<img style="float: right; margin: 30px 15px 15px 15px;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFzQj91sOlkeDFkg5HDbjtR4QJYmLXkfMNig&usqp=CAU" width="300" height="400" /> 
    
    
### <font color='navy'> Modelos no Lineales para Pronósticos. </font>

**Nombres:**
> `Cárdenas Gallardo Paula Daniela` | `733720` <br> `Haces López José Manuel` | `734759` <br> `Villa Domínguez Paulo Adrián` | `733773`

**Fecha:** Mayo 2023
    
**Profesor:** Óscar David Jaramillo Zuluaga.
    
**Link Github**: [github.com](https://github.com/paucardenasg/Proyecto_MNLP)

# <font color='maroon'> Proyecto Final </font>

In [20]:
# Librerías
import numpy as np
import pandas as pd

In [21]:
# Carga de datos
data = pd.read_csv('./Data/dataset.csv')
# Eliminando la columna de index
data.drop(columns=['SN'], inplace=True)

data

,Commodity,Date,Unit,Minimum,Maximum,Average
0,Tomato Big(Nepali),2013-06-16,Kg,35.0,40.0,37.5
1,Tomato Small(Local),2013-06-16,Kg,26.0,32.0,29.0
2,Potato Red,2013-06-16,Kg,20.0,21.0,20.5
3,Potato White,2013-06-16,Kg,15.0,16.0,15.5
4,Onion Dry (Indian),2013-06-16,Kg,28.0,30.0,29.0
...,...,...,...,...,...,...
197156,Garlic Dry Nepali,2021-05-13,Kg,100.0,120.0,110.0
197157,Fish Fresh(Rahu),2021-05-13,KG,270.0,280.0,275.0
197158,Fish Fresh(Bachuwa),2021-05-13,KG,225.0,235.0,230.0
197159,Fish Fresh(Chhadi),2021-05-13,KG,220.0,230.0,225.0


## EDA

In [22]:
# Productos
tipos_de_productos = data['Commodity'].unique()
print(f'Cantidad de Productos: {len(tipos_de_productos)}')

Cantidad de Productos: 132


In [36]:
tipos_de_unidad = data.groupby('Unit')['Commodity'].nunique().reset_index()

,Unit,Commodity
0,1 Pc,3
1,Doz,1
2,KG,13
3,Kg,115


In [35]:
data[data['Unit'].isin(['1 Pc', 'Doz'])].value_counts('Commodity')

Commodity
Banana       2744
Pineapple    2634
Sugarcane     462
Maize          24
dtype: int64

In [27]:
# Porcentaje que representa cada producto
porcentaje = data['Commodity'].value_counts(normalize=True).reset_index().sort_values(by='Commodity', ascending=False)
# Multiplicando por 100 para obtener el porcentaje
porcentaje['Commodity'] = np.round(porcentaje['Commodity'] * 100, decimals=3)
# Haciendo una suma acumulativa
porcentaje['Acumulado'] = porcentaje['Commodity'].cumsum()
porcentaje

,index,Commodity,Acumulado
0,Ginger,1.395,1.395
1,Cauli Local,1.395,2.790
2,Cabbage(Local),1.394,4.184
3,Chilli Dry,1.394,5.578
4,Raddish White(Local),1.393,6.971
...,...,...,...
127,Maize,0.012,99.974
128,Mango(Calcutte),0.012,99.986
129,Sweet Lime,0.007,99.993
130,Musk Melon,0.005,99.998


In [24]:
porcentaje.sum()

index        GingerCauli LocalCabbage(Local)Chilli DryRaddi...
Commodity                                               100.03
dtype: object